# **<center><mark> Exotics </mark> </center>**

**Background Information:**
- Exotics are options/derivatives that have features which are more complex than vanilla options <br> 
- They depend on more variables than just the stock price at Maturity. These are called path-dependent options <br>
- The payoff thus depends on what the path of the stock price leading up to its Maturity <br>
- They exist because vanilla options are too rigid to protect from specific financial risks <br>
- The options will be priced equal to their Expected Average payoffs <br>
- The list of Exotics below are not exhaustive <br>
*All calculations will be done using monte carlo simulation

<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

1. Simulating underlying price (lognormal Distribution)
$$
    dS(t) = rS(t)dt + \sigma S(t)dB(t)
$$ 
2. Geometric Brownian motion 
$$ 
    S(t + dt) = S(t){e^{(r - {\sigma ^2/2})dt + \sigma \sqrt(dt)Z_t}} \\
    lnS(t + dt) = lnS(t) + (r - {\sigma ^2/2})dt + \sigma \sqrt(dt)Z_t
$$

In [54]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, percentileofscore

# simulate prices based on geometric brownian motion
spot_price = 100
strike_price = 105
Time = 1
risk_free_rate = 0.05
sigma = 0.2
simulations = 100000 # can change number of simulations according to your CPU limits
Time_steps = 100

dt = Time / Time_steps

# rows represent simulations, columns represent timesteps
random = np.random.normal(size=(simulations, Time_steps))

# The ratio matrix represent the ratio of each price to the previous timestep 
ratio_matrix = np.exp((risk_free_rate - sigma * sigma / 2) * dt + sigma * random * np.sqrt(dt))

# Now calculate the prices using the cumulative product of the ratios
prices = spot_price * np.cumprod(ratio_matrix, axis=1)

<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Asian Options </h3></b> </center>

- The payoff of Asian options depends on the average of the price of the
underlying until its Maturity. Therefore, the payoff is far less sensitive to the
final value of the underlying. <br>
- This is useful for options which may be subject to fluctuations or market manipulations right before Maturity

- Option Payoff
$$
    payoff = max\{average(assetprice) - K\}\\ 
$$
- Variance
$$
    \frac{1}{N-1} \sum_{i=1}^n (Payoff - Average)^2
$$
- Standard Error
$$
    \sqrt \frac{Variance}{N}
$$

Where Z(t) is a standard normal r.v. <br>
      S(t) is stock price at Time t <br>
      K is option strike price <br>
      B(t) is Brownian Motion <br>

In [55]:
# Calculate the Asian call payoff per simulation, get the mean of each simulation
payoffs = np.maximum(0, np.mean(prices, axis=1) - strike_price)

# Call_price is the average of all simulations
Asian_Option_call_price = np.mean(payoffs)*np.exp(-risk_free_rate*Time)

# calculate the variance of payoffs for the Asian Option
var_matrix = np.power((payoffs - Asian_Option_call_price),2)
variance = np.sum(var_matrix)/(simulations-1)

# calculate standard error for calculation
standard_error = np.sqrt(variance/simulations)

print("Asian Option call price is: ",Asian_Option_call_price)
print("Variance of call price is: ",variance)
print("Standard error is: ",standard_error)

Asian Option call price is:  3.573752215050642
Variance of call price is:  46.793562431253534
Standard error is:  0.021631819718011133


<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Lookback Options </h3></b> </center>

- They are similar to calls or puts But are dependent on the Maximum of Minimum price of the underlying until its Maturity

- Payoff of lookback call: $$[Max(T) - S(T)]$$
- Payoff of lookback put: $$[S(T) - Min(T)]$$

In [56]:
# Calculate the Average Maximum price
payoffs = np.maximum(0,np.amax(prices - strike_price, axis=1))
Average_Max_Price = np.sum(payoffs)/simulations

# Calculate the Average Minimum price
payoffs = np.maximum(0,np.amax(strike_price - prices, axis=1))
Average_Min_Price = np.sum(payoffs)/simulations

# Calculate the payoffs of each lookback option
lookbackpayoffofcall = Average_Max_Price*np.exp(-risk_free_rate*Time)
lookbackpayoffofput = Average_Min_Price*np.exp(-risk_free_rate*Time)

print("Price of lookback call is: ",lookbackpayoffofcall)
print("Price of lookback put is: ",lookbackpayoffofput)

Price of lookback call is:  13.75615243846724
Price of lookback put is:  16.055970740060452


<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Barrier Options </h3></b> </center>

- The payoff depends on whether the price of the underlying crosses, or does not cross a certain price barrier throughout its life.
- If the underlying does not cross the price barrier, then the option expires worthless
- They are called knock-in and knock-out options

In [57]:
# example where barrier to not drop below is $99
Barrier_price = 99

# Calculate the payoff per simulation
payoffs = np.maximum(0, prices - Barrier_price)

# count rows where any payoff did not hit zero, which means the options are not knocked out
Barrier_rows = np.where(np.prod(payoffs, axis=1)!=0)[0]

# sum all the Final payoffs of Barrier_rows
Barrier_total = 0
for i in Barrier_rows:
    Barrier_total += payoffs[i][-1]

# divide over total simulations and discount using risk_free_rate
Barrier_option_price = Barrier_total/simulations*np.exp(-risk_free_rate*Time)

print("Price of Barrier Option is: ",Barrier_option_price)

Price of Barrier Option is:  3.0190143623654073


<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Basket Options </h3></b> </center>

- Basket Options pay the weighted average of the underlying in the basket

In [58]:
# Example would be 50% Asian option, 50% Barrier Option
Basket_Option_price = 0.5*Asian_Option_call_price + 0.5*Barrier_option_price

print("Price of Basket Option is: ",Basket_Option_price)

Price of Basket Option is:  3.2963832887080247


<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Binary Options </h3></b> </center>

- A binary, or digital, option pays one dollar if the underlying ends
up above a given strike price K, and zero dollars otherwise. However, the payoff is discontinuous.
- The option price based on the Merton-Black-Scholes model is:
$$e^{-rT}N(d_2)$$
where 
$$d_1 = \frac{1}{\sigma\sqrt(T-t)}[log(s/K) + (r-q+\frac{\sigma^2}{2})(T-t)$$
$$d_2 = d_1 - \sigma\sqrt(T-t)$$

In [59]:
# We assume the payoff per simulation is either $1 or $0 for simplicity
# Calculate the Payoffs per simulation
payoffs = np.maximum(0, np.mean(prices, axis=1) - strike_price)

# Count percentage of non-zero payoffs simulations
binarypayoffs = np.count_nonzero(payoffs)/simulations

# Discount the binarypayoffs to get the option price
binaryoption = binarypayoffs*np.exp(-risk_free_rate*Time)

print("Price of binary option is: ",binaryoption)

Price of binary option is:  0.37554537679288186


<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Bermudan Options </h3></b> </center>

- Bermudan options are similar to American options, except that they
can be exercised only at specific dates or during specific time intervals. 
- Mathematically, the price is determined as a maximum of expectations taken over all stopping times taking values in the set of the possible exercise times.

<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Chooser Options </h3></b> </center>

- Chooser Options allow the holder to choose whether the option will be a call or put at Maturity
- The payoff of the chooser option is as follows:
$$ Max[C(T),P(T)] = Max[C(T),C(T)+Ke^{-r(T-t)}-S(T)]$$
$$ = C(T) + Max[0,Ke^{-r(T-t)}-S(T)]$$

In [60]:
# Calculate and sum the call payoffs for each simulation
choosercallpayoffs = np.maximum(0, prices - strike_price)
sum1 = 0
for i in range(simulations):
    sum1 += choosercallpayoffs[i][-1]

# Calculate and sum the put payoffs for each simulation
sum2 = 0
chooserputpayoffs = np.maximum(0, strike_price - prices)
for i in range(simulations):
    sum2 += chooserputpayoffs[i][-1]

# Get the price of the chooser option based on the max payoff of the call and put
chooser_option_price = np.maximum(sum1,sum2)/simulations

print("Average Payoff of call option is: ", sum1/simulations)
print("Average Payoff of put option is: ", sum2/simulations)
print("Price of chooser option is: ",chooser_option_price)


Average Payoff of call option is:  8.47401550644663
Average Payoff of put option is:  8.294945833524476
Price of chooser option is:  8.47401550644663


<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Range Options </h3></b> </center>

- Range options allow the holder to choose the range of prices over which the underlying will fluctuate
- If the future price exceeds the range (e.g. +-5 of spot price), then the option will expire worthless
- Otherwise, the holder will receive a specific payoff (most likely dependent on probability of price range)

In [63]:
print("40th percentile is: ",np.percentile(prices,40))
print("50th percentile is: ",np.percentile(prices,50))
print("60th percentile is: ",np.percentile(prices,60))

percentile_of_95 = percentileofscore(prices, 95)
percentile_of_105 = percentileofscore(prices, 105)

print(percentile_of_95)
print(percentile_of_105)
# try to get probability ranges based on percentile

40th percentile is:  98.36352244827333
50th percentile is:  101.10240701454474
60th percentile is:  104.09909784500792
[0.057 0.028 0.    ... 0.014 0.047 0.031]
[0.099 0.075 0.029 ... 0.07  0.083 0.091]


<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Quanto Options </h3></b> </center>

<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">

<center><b><h3>Rainbow Options </h3></b> </center>

<hr style="height:10px;border:none;background-color:rgb(255, 255, 0);">